In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import numpy as np
import pandas as pd
from machinelearning_fedbatch import main, plot_net_predictions, validate_predictions
from system_ode_fedbatch import simulate, PlotSolution
from src.utils import get_data_and_feed, plot_experiment
import matplotlib.pyplot as plt

FILENAME = "../data/data_processed.xlsx"
EXPERIMENT = "BR01"
S_IN = 1.43 * 200

In [ ]:
full_df, feeds = get_data_and_feed(
    file_name=FILENAME, experiment=EXPERIMENT, keep_only="FB"
)

full_df['Biomass'].iloc[1] = 5.0
# add new line to full_df
new_row = pd.DataFrame([{"Process": "FB", "RTime": 5.85, "V": 1.56, "Biomass": 5.8, "Glucose": 0.013, "Protein": 0.0}])
full_df = pd.concat([full_df, new_row], ignore_index=True)
full_df.sort_values(by="RTime", inplace=True)

T_FB = full_df["RTime"].iloc[0] # Time of fed-batch
T_START = 0
T_END = full_df["RTime"].iloc[-1] - T_FB  # End of experiment

# Get initial volume
V0 = full_df["V"].iloc[0]

# Normalize time
full_df["RTime"] = full_df["RTime"] - T_FB
feeds["Time"] = feeds["Time"] - T_FB

print(f"Dataset shape: {full_df.shape}")

plot_experiment(full_df, title='')

In [ ]:
# inlet flowrate
def Fs(t):
    if t <= 4.73 - T_FB:
        return 0.017
    elif t <= 7.33 - T_FB:
        return 0.031
    elif t <= 9.17 - T_FB:
        return 0.060
    elif t <= 9.78 - T_FB:
        return 0.031
    else:
        return 0.017

# Plot Fs(t) 
Fs_t = [Fs(i) for i in np.linspace(T_START,T_END,1000)]
plt.figure(figsize=(10,3))
plt.plot(np.linspace(T_START,T_END,1000),Fs_t)
# Plot vertical lines on feed times
for feed_time in feeds["Time"]:
    plt.axvline(feed_time, color='black', alpha=0.2, linestyle='--')
plt.xlabel('Time (hours)')
plt.ylabel('Flowrate (liter/hour)')
plt.title('Inlet Flowrate')
plt.ylim(0,0.1)
plt.show()

In [ ]:
t_start = full_df["RTime"].iloc[0]
t_end = full_df["RTime"].iloc[-1]
IC = [full_df["Biomass"].iloc[0], full_df["Glucose"].iloc[0], V0]
sol_df = simulate(
    feeds,
    0.724,
    0.160,
    0.660,
    S_IN,
    t_start,
    t_end,
    NUM_SAMPLES=1000,
    IC=IC,
    return_df=True
)

plt.figure(figsize=(10, 5))
plt.scatter(full_df["RTime"], full_df["Biomass"], label="Biomass", color='green')
plt.plot(sol_df["RTime"], sol_df["Biomass"], label="_Biomass", color='green')
plt.scatter(full_df["RTime"], full_df["Glucose"], label="Glucose", color='red')
plt.plot(sol_df["RTime"], sol_df["Glucose"], label="_Glucose", color='red')
# Plot vertical lines on feed times
for feed_time in feeds["Time"]:
    plt.axvline(feed_time, color='black', alpha=0.2, linestyle='--')
plt.xlabel("Time (h)")
plt.ylabel("Concentration (g/L)")
plt.legend()
plt.show()


In [6]:
for i in range(2, len(full_df)+1):
    print(f"Training with {i} samples")
    train_df = full_df.iloc[:i]
    net, u_pred, loss = main(train_df=train_df, full_df=full_df, feeds=feeds, Sin=S_IN, V0=V0, num_epochs=30000, verbose=100)
    
    # Clip u_pred to be positive
    u_pred[u_pred < 0] = 0
    
    print(f"mu_max = {net.mu_max.item():.4f}")
    print(f"Ks = {net.K_s.item():.4f}")
    print(f"Yxs = {net.Y_xs.item():.4f}")

    title = f"mu_max = {net.mu_max.item():.4f}, Ks = {net.K_s.item():.4f}, Yxs = {net.Y_xs.item():.4f} | Loss = {loss:.4f}"
    plot_net_predictions(full_df=full_df, train_df=train_df, u_pred=u_pred, title=title)
    
    validate_predictions(full_df=full_df, u_pred=u_pred, i=i)
    
    # Save u_pred to file
    u_pred.to_csv(f"./temp/u_pred_{i}.csv", index=False)

In [ ]:
t_start = full_df["RTime"].iloc[0]
t_end = full_df["RTime"].iloc[-1]
IC = [full_df["Biomass"].iloc[0], full_df["Glucose"].iloc[0], V0]
sol_df = simulate(
    feeds,
    net.mu_max.item(),
    net.K_s.item(),
    net.Y_xs.item(),
    S_IN,
    t_start,
    t_end,
    NUM_SAMPLES=1000,
    IC=IC,
    return_df=True
)

PlotSolution(full_df=full_df, train_df=train_df, df_pred=sol_df)

In [ ]:
# Quick and dirty way to plot results
res = pd.read_csv('./temp/u_pred_12.csv')

plt.figure(figsize=(10, 5))
plt.scatter(full_df["RTime"], full_df["Biomass"], label="Biomass", color='green')
plt.plot(res["RTime"], res["Biomass"], label="_Biomass", color='green')
plt.scatter(full_df["RTime"], full_df["Glucose"], label="Glucose", color='red')
plt.plot(res["RTime"], res["Glucose"], label="_Glucose", color='red')
plt.xlabel("Time (h)")
plt.ylabel("Concentration (g/L)")
plt.legend()
plt.show()